**IFTS** TECNICO DEL DISEGNO E PROGETTAZIONE INDUSTRIALE SPECIALIZZATO IN SMART MANUFACTURING (2019)  
**INDUSTRIA 4.0 - Tecnologie Informatiche di Riferimento**

# Machine Learning

## Esercitazione 2: Classificazione

Gianluca Moro, **Roberto Pasolini**  
DISI - Dipartimento di Informatica - Scienza e Ingegneria  
Università di Bologna  
nome.cognome@unibo.it

## Esempio 3: valutazione del rischio di credito

Una banca vuole ottenere un modello per la valutazione del rischio nella concessione di prestiti. Per questo ha a disposizione un insieme di casi già etichettati come "sicuri" o "a rischio".

Si tratta di un problema di classificazione: per ogni nuovo caso, il modello deve indicare una di queste due etichette.

Iniziamo caricando i dati, che si trovano nel file CSV `bank.csv`.

In [1]:
import pandas as pd
data = pd.read_csv("bank.csv")

Vediamo il numero di righe e di colonne

In [2]:
data.shape

(637, 22)

Vediamo alcuni esempi di dati dalle prime righe (`head`) e dalle ultime (`tail`); nella visualizzazione alcune colonne sono omesse per brevità.

In [3]:
data.head(5)

,AccountStatus,Months,CreditHistory,Purpose,Amount,Savings,Employment,InstallmentRate,Sex,Status,...,Property,Age,InstallmentPlans,Housing,CreditsHere,Job,LiablePeople,Telephone,ForeignWorker,Risk
0,0 - 200 DM,13.0,Critical,Radio/television,882.0,< 100 DM,< 1 year,4.0,Male,Single,...,Real estate,23.0,None,Own,2.0,Employee,1.0,None,Yes,Good
1,>= 200 DM / salary >1yr,15.0,Some paid back,Business,2687.0,< 100 DM,4 - 7 years,2.0,Male,Single,...,Bldg. society savings / Life insurance,26.0,None,Rent,1.0,Employee,1.0,Registered,Yes,Good
2,No account,6.0,Some paid back,Domestic appliances,1338.0,500 - 1000 DM,1 - 4 years,1.0,Male,Married/Separated/Widowed,...,Real estate,62.0,None,Own,1.0,Employee,1.0,None,Yes,Good
3,No account,24.0,Critical,Car (new),1287.0,>= 1000 DM,>= 7 years,4.0,Female,Married/Separated/Widowed,...,Real estate,37.0,None,Own,2.0,Employee,1.0,Registered,Yes,Good
4,< 0 DM,21.0,Some paid back,Radio/television,2606.0,< 100 DM,< 1 year,4.0,Female,Married/Separated/Widowed,...,Bldg. society savings / Life insurance,28.0,None,Rent,1.0,Mgmt / Self-employed / other,1.0,Registered,Yes,Good


In [4]:
data.tail(5)

,AccountStatus,Months,CreditHistory,Purpose,Amount,Savings,Employment,InstallmentRate,Sex,Status,...,Property,Age,InstallmentPlans,Housing,CreditsHere,Job,LiablePeople,Telephone,ForeignWorker,Risk
632,0 - 200 DM,24.0,Some paid back,Furniture/equipment,4057.0,< 100 DM,4 - 7 years,3.0,Male,Married/Separated/Widowed,...,Car or other,43.0,None,Own,1.0,Employee,1.0,Registered,Yes,Bad
633,No account,6.0,Some paid back,Furniture/equipment,4611.0,< 100 DM,< 1 year,1.0,Female,Married/Separated/Widowed,...,Bldg. society savings / Life insurance,32.0,None,Own,1.0,Employee,1.0,None,Yes,Bad
634,0 - 200 DM,60.0,All paid back,Other,14782.0,100 - 500 DM,>= 7 years,3.0,Female,Married/Separated/Widowed,...,Unknown/None,60.0,Bank,For free,2.0,Mgmt / Self-employed / other,1.0,Registered,Yes,Bad
635,< 0 DM,36.0,Critical,Car (used),9629.0,< 100 DM,4 - 7 years,4.0,Male,Single,...,Car or other,24.0,None,Own,2.0,Employee,1.0,Registered,Yes,Bad
636,< 0 DM,48.0,Critical,Car (used),6331.0,< 100 DM,>= 7 years,4.0,Male,Single,...,Unknown/None,46.0,None,For free,2.0,Employee,1.0,Registered,Yes,Bad


La colonna "Risk" sulla destra è ciò che vogliamo predire: è "Good" per i clienti considerati sicuri e "Bad" per quelli considerati a rischio. Possiamo usare la funzione `value_counts` per vedere quanti esempi sono presenti di un tipo e dell'altro.

In [5]:
data["Risk"].value_counts()

Good    348
Bad     289
Name: Risk, dtype: int64

Per iniziare, come abbiamo fatto negli esempi con la regressione, dividiamo i dati in un training set per addestrare i modelli e in un validation set per valutarne l'accuratezza, distinguendo la variabile che vogliamo predire ("Risk") da quelle su cui vogliamo basare la predizione (tutte le altre).

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = \
    train_test_split(data.drop(columns=["Risk"]), data["Risk"], test_size=0.3, random_state=42)

## Conversione variabili categoriche

Nella tabella sono presenti due tipi di variabili: quelle _numeriche_ in cui ciascun valore è un numero (es. "Amount", "InstallmentRate", ecc.) e quelle _categoriche_ i cui valori possibili sono ristretti ad un insieme specifico (es. "Sex" può essere "Male" o "Female").

Diversi algoritmi di ML possono gestire solamente variabili di tipo numerico. Tuttavia ogni variabile di tipo categorico con N valori possibili può essere facilmente convertita in N variabili numeriche: ciascuna variabile avrà valore 1 nei casi in cui la variabile categorica ha il valore che rappresenta e 0 negli altri casi. Tale codifica delle variabili categoriche è detta _one-hot encoding_.

Definiamo una funzione che sostituisca le variabili categoriche con tale codifica.

In [7]:
def encode_categorical(df):
    return pd.concat(
        [pd.get_dummies(df[col], col) if df[col].dtype == object else df[[col]]
         for col in df], axis=1)

Applichiamola quindi alle tabelle `X_train` e `X_val`.

In [8]:
X_train = encode_categorical(X_train)
X_val = encode_categorical(X_val)

Tutte le variabili predittive (X) sono ora numeriche.

In [9]:
X_train.head(5)

,AccountStatus_0 - 200 DM,AccountStatus_< 0 DM,AccountStatus_>= 200 DM / salary >1yr,AccountStatus_No account,Months,CreditHistory_All paid back,CreditHistory_Critical,CreditHistory_No history,CreditHistory_Past delays,CreditHistory_Some paid back,...,CreditsHere,Job_Employee,Job_Mgmt / Self-employed / other,Job_Unemployed/unskilled non-resident,Job_Unskilled resident,LiablePeople,Telephone_None,Telephone_Registered,ForeignWorker_No,ForeignWorker_Yes
75,1,0,0,0,20.0,0,0,0,1,0,...,2.0,0,1,0,0,2.0,0,1,0,1
92,0,0,0,1,10.0,0,1,0,0,0,...,2.0,1,0,0,0,1.0,1,0,1,0
336,0,1,0,0,24.0,1,0,0,0,0,...,1.0,1,0,0,0,1.0,0,1,0,1
68,0,1,0,0,48.0,0,0,0,0,1,...,1.0,0,1,0,0,1.0,0,1,0,1
299,0,0,0,1,12.0,0,1,0,0,0,...,1.0,1,0,0,0,1.0,1,0,1,0


## Alberi decisionali

Gli _alberi decisionali_ sono una classe di modelli di classificazione molto usati nella pratica. Dai dati di addestramento viene costruito un diagramma ad albero, per classificare un elemento si segue un percorso dalla radice dell'albero ad una "foglia" con la classe predetta in base ai valori delle variabili. Tale modello ha il vantaggio di essere semplice da interpretare: osservando l'albero generato è immediato capire quali siano le variabili più importanti ai fini della predizione.

L'uso di modelli di classificazione in scikit-learn è simile a quello dei modelli di regressione: iniziamo caricando la classe dei modelli che vogliamo usare.

In [10]:
from sklearn.tree import DecisionTreeClassifier

Inizializziamo quindi un modello definendone i parametri: in questo caso usiamo `max_depth` che limita la profondità dell'albero e `random_state` che fissa le scelte casuali.

In [11]:
model = DecisionTreeClassifier(max_depth=10, random_state=123)

Per addestrare il modello usiamo la funzione `fit` passando le variabili predittive (X) e i valori da prevedere (y) come nella regressione, con la differenza che quì i valori y sono le etichette "Good" e "Bad" invece di numeri.

In [12]:
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

Il modello che si ottiene è simile a questo:

![albero decisionale](tree.svg)

Per classificare un esempio si parte dal nodo radice in alto, quindi in ogni nodo si verifica il valore della variabile indicata (ad es. `AccountStatus_NoAccount` nella radice) e si segue il ramo sinistro o destro a seconda che sia sotto o sopra la soglia indicata. Si prosegue fino a raggiungere un nodo foglia (quelli in basso), che indica la classe predetta. Le variabili che compaiono ai livelli superiori sono in genere quelle più d'aiuto nel distinguere gli esempi delle diverse classi.

Per ottenere predizioni dal modello possiamo usare `predict` come per i modelli di regressione. Otteniamo ad esempio le predizioni relative ai primi 5 esempi del validation set.

In [13]:
model.predict(X_val.head(5))

array(['Bad', 'Bad', 'Good', 'Bad', 'Bad'], dtype=object)

I modelli di classificazione forniscono anche la funzione `predict_proba`, che indica le probabilità stimate dal modello per ciascuna classe, in caso sia necessario distinguere le predizioni più sicure da quelle più incerte.

In [14]:
pd.DataFrame(model.predict_proba(X_val.head(5)), columns=model.classes_)

,Bad,Good
0,1.000000,0.000000
1,1.000000,0.000000
2,0.000000,1.000000
3,0.983333,0.016667
4,0.983333,0.016667


Ad es. la predizione "Bad" sul primo esempio è data certa (100%), mentre sul terzo c'è una minima incertezza (98%).

Per valutare l'accuratezza del modello anche in questo caso possiamo usare la funzione `score` passando i dati del validation set. Nel caso della classificazione, la metrica di accuratezza è la percentuale di casi del validation set che vengono classificati correttamente.

In [15]:
model.score(X_val, y_val)

0.6041666666666666

Significa che circa il 60% degli esempi è classificato correttamente.

Un'indicazione più specifica sull'efficacia del modello può essere data dalla _matrice di confusione_, che indica il numero di esempi per ogni combinazione possibile di classe reale e classe predetta. Per estrarre tale matrice usiamo la funzione `confusion_matrix` passando la serie delle classi reali del validation `y_val` e quelle predette dal modello, ottenute tramite `predict`. L'ultima riga del codice sotto serve a visualizzare la matrice in modo comprensibile con righe e colonne etichettate.

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, model.predict(X_val), labels=model.classes_)
pd.DataFrame(cm, index=model.classes_, columns=model.classes_)

,Bad,Good
Bad,47,33
Good,43,69


Questa matrice indica ad esempio che, delle 80 istanze "Bad" del validation set, 47 sono state effettivamente etichettate come "Bad" mentre 33 sono state erroneamente etichettate "Good". In pratica il modello ha erroneamente previsto come "sicuri" 33 clienti a rischio.

Considerare la matrice di confusione invece della sola accuratezza è molto importante nel caso in cui gli errori abbiano costi differenti. Ad esempio una banca potrebbe perdere molti più soldi concedendo un prestito che non viene restituito piuttosto che non concedendone uno che verrebbe restituito, per cui può cercare di ottenere un modello che minimizzi gli errori del primo tipo considerando meno gravi gli altri.

Addestriamo un altro modello diminuendo il parametro `max_depth`, quindi limitando ancora di più la profondità dell'albero.

In [17]:
model = DecisionTreeClassifier(max_depth=5, random_state=123)
model.fit(X_train, y_train)
model.score(X_val, y_val)

0.671875

In [18]:
cm = confusion_matrix(y_val, model.predict(X_val), labels=model.classes_)
pd.DataFrame(cm, index=model.classes_, columns=model.classes_)

,Bad,Good
Bad,53,27
Good,36,76


Questo secondo albero decisionale compie meno errori del primo nonostante sia più semplice: può essere segno di _overfitting_ nel primo albero.

## Support Vector Machine

Un altro esempio di modello di classificazione sono le _Support Vector Machine_ (SVM); tali modelli astraggono i dati come punti in uno spazio multidimensionale e cercano il piano che divide in modo più netto possibile i punti di una classe da quelli dell'altra. Come la regressione kernel ridge, SVM si basa su funzioni kernel per riuscire a separare classi anche in modo non lineare.

La classe `SVC` implementa i modelli di classificazione tramite SVM (esistono anche modelli SVM per la regressione). Il funzionamento è simile agli altri...

In [19]:
from sklearn.svm import SVC
model = SVC(gamma="auto")
model.fit(X_train, y_train)
model.score(X_val, y_val)

0.5729166666666666

Questo modello è meno accurato del precedente basato su albero decisionale, ma SVM è generalmente più efficace nella classificazione di testi, come vediamo di seguito.

## Esempio 4: classificazione di commenti positivi e negativi

Vediamo ora come creare un modello che riconosca commenti positivi o negativi addestrandolo su recensioni già etichettate.

Il file all'URL `https://git.io/vpaDt` contiene 10.000 recensioni di film estratte da Amazon, etichettate col numero di stelle dato dall'utente da 1 a 5.

In [20]:
reviews = pd.read_csv("https://git.io/vpaDt", sep="\t", compression="gzip")

Vediamo alcune di queste recensioni. La prima riga serve ad aumentare il numero massimo di caratteri mostrato.

In [21]:
pd.options.display.max_colwidth = 100
reviews.tail(6)

,text,stars
9994,Dudley Boyz VS Ric Flair and Batista: This match was rather short. Dudley's looked good but Coac...,4
9995,"You seen one heist film, you seen them all. But every once in a while, somebody who really gives...",5
9996,"Often compared with ""The Big Chill"", and getting numerous stars in many reviews, this film simpl...",1
9997,This collection of Laurel and Hardy films contains five total selections. Four of these are sho...,3
9998,I love Vin Diesel but I wish I'd skipped this movie. The first bad sign was the fact that this t...,3
9999,"When The Office was first shown to a UK audience back in 2001, it was shown on BBC2. That is the...",5


Per dividere le recensioni in due classi, etichettiamo come "positive" quelle con 4 o 5 stelle e come "negative" le altre con 3 stelle o meno. Aggiungiamo alla tabella una colonna "etichetta" che valga "pos" o "neg" a seconda di questa condizione.

In [22]:
reviews["label"] = pd.np.where(reviews["stars"] >= 4, "pos", "neg")

In [23]:
reviews.tail(6)

,text,stars,label
9994,Dudley Boyz VS Ric Flair and Batista: This match was rather short. Dudley's looked good but Coac...,4,pos
9995,"You seen one heist film, you seen them all. But every once in a while, somebody who really gives...",5,pos
9996,"Often compared with ""The Big Chill"", and getting numerous stars in many reviews, this film simpl...",1,neg
9997,This collection of Laurel and Hardy films contains five total selections. Four of these are sho...,3,neg
9998,I love Vin Diesel but I wish I'd skipped this movie. The first bad sign was the fact that this t...,3,neg
9999,"When The Office was first shown to a UK audience back in 2001, it was shown on BBC2. That is the...",5,pos


Vediamo la distribuzione tra recensioni positive e negative.

In [24]:
reviews["label"].value_counts()

pos    7328
neg    2672
Name: label, dtype: int64

## Rappresentazione bag of words

Per addestrare un modello ML su dei testi è necessario estrarre delle variabili da di essi. Per questo è possibile rappresentare ogni testo in forma di un _bag of words_, ovvero un insieme delle parole contenute in ciascun testo con associato il loro numero di occorrenze. In questo modo da ogni parola distinta (_termine_) presente nei testi è estratta una variabile su cui il modello può apprendere.

scikit-learn fornisce la classe `CountVectorizer` per convertire ciascun testo di un insieme dato in un vettore di conteggi delle parole presenti.

Per vedere come funziona, creiamo una breve lista di frasi d'esempio da processare.

In [25]:
examples = [
    "the sky is blue",
    "sky is blue and sky is beautiful",
    "the beautiful sky is so blue",
    "i love blue cheese"
]

Per convertire tali frasi in bag of words, creiamo prima un oggetto `CountVectorizer`.

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

A questo punto passiamo la lista di frasi a tale oggetto usando la funzione `fit_transform`.

In [27]:
dtm = vect.fit_transform(examples)

L'oggetto `dtm` ottenuto è una _matrice documenti-termini_, che contiene una riga per ogni documento e una colonna per ogni termine: ogni suo valore indica il numero di occorrenze di un termine in un documento. Visualizziamo tale matrice...

In [28]:
pd.DataFrame(dtm.toarray(), columns=vect.get_feature_names())

,and,beautiful,blue,cheese,is,love,sky,so,the
0,0,0,1,0,1,0,1,0,1
1,1,1,1,0,2,0,2,0,0
2,0,1,1,0,1,0,1,1,1
3,0,0,1,1,0,1,0,0,0


La matrice indica ad esempio che la seconda frase ("sky is blue and sky is beautiful") contiene tra le altre la parola "sky" 2 volte e la parola "blue" 1 volta. I termini utilizzati come variabili sono tutti quelli presenti nei testi passati alla funzione `fit_transform`. Data una nuova frase d'esempio...

In [29]:
new_example = "loving this blue sky today"

...è possibile usare la funzione `transform` per estrarne il bag of words mantenendo inalterato il set di variabili.

In [30]:
pd.DataFrame(vect.transform([new_example]).toarray(), columns=vect.get_feature_names())

,and,beautiful,blue,cheese,is,love,sky,so,the
0,0,0,1,0,0,0,1,0,0


Si noti che alcune parole della nuova frase (es. "loving") vengono "perse", in quanto non inizialmente previste tra le variabili. Quando si addestra un modello ML vanno fissate le variabili da utilizzare, per cui qualsiasi termine non presente nei testi usati per addestrare il modello sarà ignorato se trovato nei testi da classificare in seguito.

Applichiamo ora `CountVectorizer` ai dati effettivi. Per iniziare dividiamo come al solito i dati a disposizione in training e validation set, individuando come variabile X il testo della recensione e come y l'etichetta "pos" o "neg".

In [31]:
X_train, X_val, y_train, y_val = \
    train_test_split(reviews["text"], reviews["label"], test_size=0.3, random_state=42)

Creiamo ora un nuovo oggetto `CountVectorizer` per questi dati.

In [32]:
vect = CountVectorizer()

Applichiamo prima `fit_transform` ai testi del training set: questo determinerà quali termini saranno usati come variabili e ci restituirà i bag of words delle recensioni di training.

In [33]:
dtm_train = vect.fit_transform(X_train)

Quindi applichiamo `transform` ai testi del validation set: i bag of words di questi saranno basati sulle sole parole viste nel training set, in modo che siano compatibili col modello.

In [34]:
dtm_val = vect.transform(X_val)

## Addestramento modelli

Possiamo ora addestrare un modello di classificazione basato sui bag of words utilizzando gli stessi algoritmi visti sopra. Generiamo ad esempio un albero decisionale.

In [35]:
model = DecisionTreeClassifier(random_state=123)

L'albero verrà quindi addestrato sui bag of words delle recensioni del training set e sulle relative etichette "pos"/"neg".

In [36]:
model.fit(dtm_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

A questo punto, dato un generico commento da classificare...

In [37]:
my_comment = "A really wonderful movie!"

...possiamo convertirlo in bag of words...

In [38]:
my_comment_bag = vect.transform([my_comment])

...per passarlo al modello e ottenere l'orientamento stimato.

In [39]:
model.predict(my_comment_bag)

array(['pos'], dtype=object)

Come visto sopra, possiamo anche ottenere le probabilità delle due classi.

In [40]:
model.predict_proba(my_comment_bag)

array([[0., 1.]])

Il modello è quindi sicuro al 100% che il commento sia positivo.

Come al solito, possiamo effettuare il calcolo dell'accuratezza sul validation set.

In [41]:
model.score(dtm_val, y_val)

0.7003333333333334

Il modello etichetta correttamente circa il 70% delle recensioni di validazione.

Testiamo ora invece un modello SVM.

In [42]:
model = SVC(gamma="auto")
model.fit(dtm_train, y_train)
model.score(dtm_val, y_val)

0.7313333333333333

Abbiamo ottenuto un leggero incremento dell'accuratezza.

## Riduzione del numero di variabili

L'addestramento di modelli sui bag of words delle recensioni ha richiesto molto più tempo rispetto a quello per addestrare modelli simili su dati strutturati. Questo è dovuto all'elevato numero di variabili usate per rappresentare le recensioni. Per vedere tale numero, vediamo quanti termini distinti vengono riconosciuti dall'oggetto `vect`.

In [43]:
len(vect.get_feature_names())

51772

È possibile ridurre questo numero di variabili - e di conseguenza il tempo necessario per l'addestramento - senza un crollo nell'accuratezza del modello?

`CountVectorizer` accetta diverse opzioni per cambiare l'insieme di variabili. Una di queste è `min_df`, che fa sì che le parole che appaiono in un numero di testi inferiore alla soglia data sia scartata.

Ripetiamo ad esempio il processo di generazione dei bag of words e addestramento modello selezionando solo le parole che appaiono in almeno 3 recensioni: creiamo un nuovo `CountVectorizer` opportunamente configurato.

In [44]:
vect = CountVectorizer(min_df=3)

Processiamo i testi di training e validation set.

In [45]:
dtm_train = vect.fit_transform(X_train)
dtm_val = vect.transform(X_val)

Il numero di variabili considerate questa volta è...

In [46]:
len(vect.get_feature_names())

21063

Addestriamo e valutiamo un nuovo modello basato su questo insieme più ristretto di variabili.

In [47]:
model = SVC(gamma="auto")
model.fit(dtm_train, y_train)
model.score(dtm_val, y_val)

0.7313333333333333

L'accuratezza è rimasta invariata nonostante il numero di variabili si sia più che dimezzato.

Un altro accorgimento per ridurre le variabili è la rimozione delle _stopword_, ovvero parole come articoli e congiunzioni ("e", "ma", "il", ...) che prese da sole in genere non forniscono alcuna informazione sul contenuto del testo. scikit-learn include una lista di stopword inglesi, si può impostare `CountVectorizer` in modo che le rimuova con l'opzione `stop_words="english"`.

In [48]:
vect = CountVectorizer(min_df=3, stop_words="english")
dtm_train = vect.fit_transform(X_train)
dtm_val = vect.transform(X_val)
len(vect.get_feature_names())

20757

In [49]:
model = SVC(gamma="auto")
model.fit(dtm_train, y_train)
model.score(dtm_val, y_val)

0.7313333333333333

Anche la rimozione delle stopword ha ridotto il numero di feature senza far calare l'accuratezza.

Esistono molte altre tecniche per rendere la classificazione di testi più accurata e/o più efficiente, tra cui l'uso del _tf.idf_ per valutare meglio l'importanza delle parole nei testi e dello _stemming_ per raggruppare in uno stesso termine parole con la stessa radice morfologica.

## Esercizi suggeriti

- Testare diverse combinazioni di parametri di `CountVectorizer` con diversi modelli di classificazione
- Testare i modelli con frasi specifiche cercando di capire in quali casi sbagliano: ad esempio, non considerando l'ordine delle parole, è plausibile che singole frasi con negazioni come "not a bad movie" non vengano comprese